In [27]:
import h2o
from h2o.automl import H2OAutoML

# Start the H2O cluster (locally)
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 hours 16 mins
H2O_cluster_timezone:,America/Asuncion
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.1
H2O_cluster_version_age:,1 month and 13 days
H2O_cluster_name:,H2O_from_python_matiaslopez_sbpkvc
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,15.09 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [28]:
import pandas as pd
y = "Protestas"
balanced_df = pd.read_csv('../balanced_output_35.csv', index_col=False)
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/automl.html
# from sklearn.model_selection import train_test_split
# train_data, test_data = train_test_split(balanced_df, test_size=0.2, random_state=42)

## Random undersampling

In [29]:
# Random undersampling
if True:
	balanced_df_true = balanced_df[balanced_df[y] == True]
	balanced_df_false = balanced_df[balanced_df[y] == False]
	n_cmp = len(balanced_df_true)
	n_no_cmp = len(balanced_df_false)
	total = n_cmp + n_no_cmp
	sample_false = balanced_df_false.sample(int(total*.65)) #Sample 70% no complaints 30 % complaints
	balanced_df = pd.concat([balanced_df_true, sample_false])

## Training

In [30]:
train = h2o.H2OFrame.from_python(balanced_df)
# Identify predictors and response
x = train.columns
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [31]:
# Run AutoML for 10 base models
aml = H2OAutoML(max_models=42, seed=23, stopping_metric="RMSE", sort_metric="RMSE")
aml.train(x=x, y=y, training_frame=train)

# View the AutoML Leader
# board
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

AutoML progress: |
15:04:44.992: _train param, Dropping bad and constant columns: [contracts.investmentProjects.id_37, parties.details.legalEntityTypeDetail buyer_15, parties.details.legalEntityTypeDetail buyer_14, parties.details.legalEntityTypeDetail buyer_13, parties.details.legalEntityTypeDetail buyer_12, parties.details.legalEntityTypeDetail buyer_11, parties.details.legalEntityTypeDetail buyer_10, parties.details.EntityType payee_1, parties.details.EntityType payee_2, parties.details.EntityType payee_3, parties.details.EntityType payee_4, planning.items.classification.id.n1_1_56, planning.items.classification.id.n1_1_57, contracts.investmentProjects.id_102, contracts.investmentProjects.id_100, Monto faltante, parties.details.legalEntityTypeDetail notifiedSupplier_18, tender.status_active, contracts.investmentProjects.id_17, tender.statusDetails_En Convocatoria (Abierta), parties.details.EntityType candidate_2, parties.details.EntityType candidate_1, parties.details.EntityType can

model_id,rmse,auc,logloss,aucpr,mean_per_class_error,mse
StackedEnsemble_AllModels_1_AutoML_2_20230803_150444,0.170806,0.983556,0.111544,0.976213,0.0560647,0.0291745
StackedEnsemble_BestOfFamily_1_AutoML_2_20230803_150444,0.171583,0.983267,0.112723,0.975831,0.0564435,0.0294408
GBM_grid_1_AutoML_2_20230803_150444_model_8,0.173465,0.982336,0.11878,0.974915,0.0579127,0.03009
GBM_grid_1_AutoML_2_20230803_150444_model_7,0.173857,0.98225,0.119337,0.974899,0.0570869,0.0302262
GBM_grid_1_AutoML_2_20230803_150444_model_5,0.174133,0.981957,0.117933,0.974272,0.0568691,0.0303222
XGBoost_grid_1_AutoML_2_20230803_150444_model_5,0.175801,0.980532,0.126829,0.972944,0.0587378,0.030906
GBM_1_AutoML_2_20230803_150444,0.17686,0.981048,0.121488,0.973018,0.0578978,0.0312794
GBM_4_AutoML_2_20230803_150444,0.17772,0.981122,0.121367,0.972978,0.0584578,0.0315844
GBM_grid_1_AutoML_2_20230803_150444_model_4,0.177743,0.980646,0.12507,0.97253,0.0573703,0.0315925
GBM_grid_1_AutoML_2_20230803_150444_model_1,0.179352,0.980849,0.124077,0.972135,0.0577016,0.0321671


In [32]:
model_path = h2o.save_model(model=aml.leader, path="train2021/smoteModelv7", force=True)

In [33]:
print(model_path)

/Users/matiaslopez/Documents/tesis/tesis-model/Training/train2021/smoteModelv7/StackedEnsemble_AllModels_1_AutoML_2_20230803_150444


In [34]:
saved_model = h2o.load_model(model_path)

## Testing 2022 Data

In [35]:
df_2022 = pd.read_csv('../df_dummizado_from_2021.csv', index_col=False)
df_2022 = df_2022[df_2022['date.year'] == 2022]
test_2022 = h2o.H2OFrame.from_python(df_2022)
test_2022[y] = test_2022[y].asfactor()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [36]:
def get_performance(model, test: h2o.H2OFrame):
	cm = model.model_performance(test).confusion_matrix().to_list()
	perf_t = cm[1][1] / (cm[1][0] + cm[1][1])
	perf_f = cm[0][0] / (cm[0][1] + cm[0][0])
	return perf_t, perf_f

In [37]:
best = ''
best_score_t = 0
best_score_f = 0
df_leaderboard = aml.leaderboard.as_data_frame()
model_ids = df_leaderboard['model_id']
for model_id in model_ids:
	model = h2o.get_model(model_id)
	score_t, score_f = get_performance(model, test_2022)
	print(model_id, score_t, score_f)
	if score_t > best_score_t and score_f > best_score_f:
		best_score_t = score_t
		best_score_f = score_f
		best = model_id

print(best, best_score_t, best_score_f)

StackedEnsemble_AllModels_1_AutoML_2_20230803_150444 0.5241109101868595 0.9201221472195436
StackedEnsemble_BestOfFamily_1_AutoML_2_20230803_150444 0.48523206751054854 0.9347476695596272
GBM_grid_1_AutoML_2_20230803_150444_model_8 0.5033152501506932 0.8979427836708453
GBM_grid_1_AutoML_2_20230803_150444_model_7 0.498794454490657 0.9097289188899603
GBM_grid_1_AutoML_2_20230803_150444_model_5 0.4804098854731766 0.9063537983499411
XGBoost_grid_1_AutoML_2_20230803_150444_model_5 0.5334538878842676 0.8992821172184721
GBM_1_AutoML_2_20230803_150444 0.4843279083785413 0.924461587913854
GBM_4_AutoML_2_20230803_150444 0.47166968053044 0.8957998499946427
GBM_grid_1_AutoML_2_20230803_150444_model_4 0.5216998191681737 0.9093003321547198
GBM_grid_1_AutoML_2_20230803_150444_model_1 0.5012055455093429 0.9208186006643094
XGBoost_grid_1_AutoML_2_20230803_150444_model_7 0.4972875226039783 0.8816564877317047
XGBoost_grid_1_AutoML_2_20230803_150444_model_1 0.5443037974683544 0.8750133933354762
XGBoost_grid

In [57]:
best_model = h2o.get_model('DeepLearning_grid_1_AutoML_2_20230803_150444_model_3')
best_model.model_performance(test_2022).confusion_matrix()

,False,True,Error,Rate
False,16280.0,2386.0,0.1278,(2386.0/18666.0)
True,1418.0,1900.0,0.4274,(1418.0/3318.0)
Total,17698.0,4286.0,0.173,(3804.0/21984.0)


In [53]:
best_model.model_performance(test_2022).confusion_matrix().to_list()

[[15527, 3139], [1102, 2216]]

In [58]:
model_path = h2o.save_model(model=best_model, path="train2021/smoteModelv8", force=True)

In [55]:
saved_model = h2o.load_model("/Users/matiaslopez/Documents/tesis/tesis-model/Training/train2021/smoteModelv8/DeepLearning_grid_1_AutoML_2_20230803_00151_model_3")

In [56]:
saved_model

Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_grid_1_AutoML_2_20230803_00151_model_3


Status of Neuron Layers: predicting Protestas, 2-class classification, bernoulli distribution, CrossEntropy loss, 38,952 weights/biases, 603.6 KB, 4,600,125 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate              rate_rms                momentum    mean_weight            weight_rms          mean_bias             bias_rms
--  -------  -------  ----------------  ---------  ----  ----  ---------------------  ----------------------  ----------  ---------------------  ------------------  --------------------  ------------------
    1        776      Input             10.0
    2        50       RectifierDropout  50.0       0.0   0.0   0.20461730867483416    0.34151124954223633     0.0         0.18682926992339283    0.7766852378845215  0.5849995765635401    0.4228494167327881
    3        2        Softmax                      0.0   0.0   0.0001095066020025115  0.00010655305231921375  0.0         -0.024883554093539716  0.68442702293396    -0.46365044840961833  1.7344741821289062

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.6747781655244295
RMSE: 0.8214488210013022
LogLoss: 4.551511407317617
Mean Per-Class Error: 0.1238022392577428
AUC: 0.8944822404172239
AUCPR: 0.7028411925709815
Gini: 0.7889644808344478

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999525343037174
       False    True    Error    Rate
-----  -------  ------  -------  ---------------
False  6079     984     0.1393   (984.0/7063.0)
True   311      2561    0.1083   (311.0/2872.0)
Total  6390     3545    0.1303   (1295.0/9935.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999953     0.798192  1
max f2                       0.999953     0.851793  1
max f0point5                 0.999953     0.750938  1
max accuracy                 0.999953     0.869653  1
max precision                0.999997     0.72585   0
max recall                   0.795635     1         324
max specificity              0.999997     0.871018  0
max absolute_mcc             0.999953     0.711991  1
max min_per_class_accuracy   0.999953     0.860682  1
max mean_per_class_accuracy  0.999953     0.876198  1
max tns                      0.999997     6152      0
max fns                      0.999997     460       0
max fps                      1.34672e-05  7063      399
max tps                      0.795635     2872      324
max tnr                      0.999997     0.871018  0
max fnr                      0.999997     0.160167  0
max fpr                      1.34672e-05  1         399
max tpr                      0.795635     1         324

Gains/Lift Table: Avg response rate: 28.91 %, avg score: 97.67 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0663312                   1                  3.34903    3.34903            0.968134         1         0.968134                    1                   0.222145        0.222145                   234.903   234.903            0.219172
2        0.100151                    1                  3.3666     3.35496            0.973214         1         0.969849                    1                   0.113858        0.336003                   236.66    235.496            0.331755
3        0.150075                